In [1]:
import json
import requests

import numpy as np

In [8]:
URL = 'http://ltdemos.informatik.uni-hamburg.de/cam-api'

proxies = {
  "http": "http://165.225.66.34:10015/",
  "https": "https://165.225.66.34:10015/",
}

def get_response(first_object, second_object, fast_search=True, 
               aspects=None, weights=None):
    num_aspects = len(aspects) if aspects is not None else 0
    num_weights = len(weights) if weights is not None else 0
    if num_aspects != num_weights:
        raise ValueError(
            "Number of weights should be equal to the number of aspects")
    params = {
        'objectA': first_object,
        'objectB': second_object,
        'fs': str(fast_search).lower()
    }
    if num_aspects:
        params.update({'aspect{}'.format(i + 1): aspect 
                       for i, aspect in enumerate(aspects)})
        params.update({'weight{}'.format(i + 1): weight 
                       for i, weight in enumerate(weights)})
    print ("get url")
    response = requests.get(url=URL, params=params, proxies=proxies)
    return response

In [9]:
'''
Example 
http://ltdemos.informatik.uni-hamburg.de/cam-api?fs=false&objectA=dog&objectB=cat&aspect1=size&weight1=3&aspect2=food&weight2=1
'''

response = get_response('dog', 'cat', False, ['size', 'food'], [3, 1])

get url


In [10]:
response_json = response.json()

In [13]:
response_json

{'extractedAspectsObject1': ['easier for it to demonstrate its intelligence',
  'poop',
  'better for you',
  'acids',
  'dogs',
  'longer',
  'easier to write about a dog',
  'zeke',
  'cats',
  'easier to identify'],
 'extractedAspectsObject2': ['easier to take care of',
  'smarter',
  'greater',
  'quicker',
  'safer',
  'lighter',
  'bigger',
  'bit',
  'cleverer',
  'baby'],
 'object1': {'name': 'dog',
  'points': {'food': 25.592407001895587,
   'none': 150.0627907878915,
   'size': 2.482540079277698},
  'sentences': [{'CAM_score': 2.482540079277698,
    'ES_score': 24.41371,
    'confidence': 1,
    'context_aspects': ['greater'],
    'id_pair': {'http://www.womansday.com/life/pet-care/news/a45352/study-decides-who-is-smarter-a-dogs-or-cats-1342/': 5},
    'text': 'A study that analyzed data of the brain versus the body size of both animals, found that the brain of the pack-animal dog has evolved at a much greater rate than the brain of the less-social cat.'},
   {'CAM_score': 1.

In [11]:
filename = "cat_dog_size_3_food_1.json"

with open(filename, 'w+') as f:
    json.dump(response_json, f)

In [7]:
response_json.keys()

dict_keys(['extractedAspectsObject1', 'extractedAspectsObject2', 'object1', 'object2', 'sentenceCount', 'winner'])

#### Json structure:
```
{
    'extractedAspectsObject1' - list with aspects for object1, 
    'extractedAspectsObject2' - list with aspects for object2, 
    'object1': {
        'name',
        'points',
        'sentences': [{
            "CAM_score", 
            "ES_score", 
            "confidence", 
            "context_aspects": ["greater"], 
            "id_pair": {
                "http://www.womansday.com/life/pet-care/news/a45352/study-decides-who-is-smarter-a-dogs-or-cats-1342/": 5
             }, 
             "text": "A study that analyzed data of the brain versus the body size of both animals, found that the brain of the pack-animal dog has evolved at a much greater rate than the brain of the less-social cat."            
        }]
    }, 
    'object2', 
    'sentenceCount', 
    'winner'
}
```

In [83]:
'''
'{} is better than {} because it is {}'
''' 

def get_first_template(comparison_result):
    template = '{} is better than {} because it is {}'
    
    winner = comparison_result['winner']
    winner_object_number = 1 if comparison_result['object1']['name'] == winner else 2
    loser_object_number = 3 - winner_object_number
    loser = comparison_result['object' + str(loser_object_number)]['name']
    aspect = comparison_result['extractedAspectsObject' + str(winner_object_number)][0] 
    
    return template.format(winner.capitalize(), loser, aspect)

In [84]:
get_first_template(response_json)

'Cat is better than dog because it is easier to take care of'

In [95]:
'''
'Both objects have some advantages: {} is {}, than {} is {}. But I would prefer {}'
'''

def get_second_template(comparison_result):
    template = 'Both objects have some advantages: {} is {}, than {} is {}. But I would prefer {}'
    return template.format(
        comparison_result['object1']['name'],
        ', '.join(comparison_result['extractedAspectsObject1']),
        comparison_result['object2']['name'],
        ', '.join(comparison_result['extractedAspectsObject2']),
        comparison_result['winner']
    )

In [ ]:
# '''
# 'There are some aspects to pay attention to while making your choice:  I would prefer {} to {}'
# '''

# def get_fourth_template(first_object, second_object, comparison_result):
    
#     return SECOND_TEMPLATE.format(
#         comparison_result['object1']['name'],
#         ', '.join(comparison_result['extractedAspectsObject1']),
#         comparison_result['object2']['name'],
#         ', '.join(comparison_result['extractedAspectsObject2']),
#         comparison_result['winner'],
#         second_object if comparison_result['winner'] == first_object else first_object
#     )

In [96]:
get_second_template(response_json)

'Both objects have some advantages: dog is easier for it to demonstrate its intelligence, poop, better for you, acids, dogs, longer, easier to write about a dog, zeke, cats, easier to identify, than cat is easier to take care of, smarter, greater, quicker, safer, lighter, bigger, bit, cleverer, baby. But I would prefer cat'

In [88]:
from tqdm import tqdm_notebook

In [52]:
pairs_without_aspects = [
    ('dog', 'cat'),
    ('cuda', 'opencl'),
    ('ios', 'android'),
    ('potato', 'steak'),
    ('pepsi', 'coca-cola'),
    ('milk', 'beer'),
    ('php', 'javascript'),
    ('python', 'java'),
    ('perl', 'python'),
]

In [53]:
results_for_pairs = {}
for pair in tqdm_notebook(pairs_without_aspects):
    comparison_result = get_response(*pair, False).json()
    results_for_pairs[pair] = comparison_result

In [97]:
for pair in pairs_without_aspects:
    comparison_result = results_for_pairs[pair]
    print(get_first_template(comparison_result))
    print(get_second_template(comparison_result))
    print("")

Cat is better than dog because it is easier to take care of
Both objects have some advantages: dog is easier for it to demonstrate its intelligence, poop, better for you, acids, dogs, longer, easier to write about a dog, zeke, cats, easier to identify, than cat is easier to take care of, smarter, greater, safer, lighter, quicker, bit, cleverer, baby, litter. But I would prefer cat

Cuda is better than opencl because it is easier for the developer
Both objects have some advantages: cuda is easier for the developer, faster, thats, easier to learn, nature, easier to use, higher, greater, ofcourse, easier to program, than opencl is easier to program for, simpler to target cuda, simpler, earlier, older, performance. But I would prefer cuda

Android is better than ios because it is easier to setup the actual tools i ve found but once thats done it s easier to code
Both objects have some advantages: ios is richer, phone, windows, state, webos, smoother, apps, malware, bigger, easier to use, t

In [100]:
'''
I would prefer {} to {} because it is {}. The argument which proved it was on {}: \"{}\"
'''

def find_sentences_with_aspect(aspect, sentences):
    sentences = np.array(sentences)
    indices = np.where([aspect in sentence["context_aspects"] for sentence in sentences])[0]
    return sentences[indices]

    
def get_third_template(comparison_result):
    winner = comparison_result['winner']
    winner_object_number = 1 if comparison_result['object1']['name'] == winner else 2
    loser_object_number = 3 - winner_object_number
    loser = comparison_result['object' + str(loser_object_number)]['name']
    aspect = comparison_result['extractedAspectsObject' + str(winner_object_number)][0] 
    winner_sentences = comparison_result['object' + str(winner_object_number)]['sentences']    
    sentence = find_sentences_with_aspect(aspect, winner_sentences)[0]
    
    first_word = aspect.split(' ')[0]
    is_comparative = (first_word[-2:] == "er")
    if is_comparative:
        template = "I would prefer {} to {} because it is {}. " 
    else:
        template = "I would prefer {} to {} because its {}. "
    
    template += "The argument which proved it was on {}: \"{}\"" 
        
    url = next(iter(sentence['id_pair']))  
    
    return template.format(
        winner,
        loser,
        aspect,
        url,
        sentence['text']
    )
    

In [101]:
get_third_template(response_json)

'I would prefer cat to dog because it is easier to take care of. The argument which proved it was on https://www.reddit.com/r/aww/comments/q57yb/most_people_have_to_fight_cats_off_their_laptops/: "I would say harder to take care of than a cat, but easier than a dog that you need to walk several times a day."'

In [102]:
for pair in pairs_without_aspects:
    comparison_result = results_for_pairs[pair]
    print(get_third_template(comparison_result))
    print("")

I would prefer cat to dog because it is easier to take care of. The argument which proved it was on https://www.reddit.com/r/aww/comments/q57yb/most_people_have_to_fight_cats_off_their_laptops/: "I would say harder to take care of than a cat, but easier than a dog that you need to walk several times a day."

I would prefer cuda to opencl because it is easier for the developer. The argument which proved it was on http://streamcomputing.eu/blog/2010-04-22/difference-between-cuda-and-opencl/: "In short CUDA certainly has made a lot of things just easier for the developer, but OpenCL has its potential in support for more than just GPUs."

I would prefer android to ios because it is easier to setup the actual tools i ve found but once thats done it s easier to code. The argument which proved it was on http://crackberry.com/comment/1066683: "BB is harder to setup the actual tools I've found but once thats done it's easier to code than Android and provides better integration and power than iO